Ukryte modele Markowa
https://medium.com/analytics-vidhya/hmm-for-multidimensional-categorical-data-68b1091c9d66

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import pandas as pd

from ezc3d import c3d
from pyomeca import Analogs, Markers
from pathlib import Path
from math import sqrt
from tqdm import tqdm


import dataprocesing as dp

In [2]:
data_path=r"Z:\baza\2020-08-06-B0445-S01-E01-T01-K02.c3d"

markers_names=['B0445:LUPA','B0445:RWRA']

markers=Markers.from_c3d(data_path, usecols=markers_names)

In [3]:
from hmmlearn import hmm

In [4]:
print(markers)

<xarray.DataArray 'markers' (axis: 4, channel: 2, time: 361)>
array([[[-6.83307373e+02, -6.84105591e+02, -6.84922058e+02, ...,
         -6.00819031e+02, -5.99556824e+02, -5.98348022e+02],
        [-2.14012421e+02, -2.14140884e+02, -2.14264771e+02, ...,
         -1.88783081e+02, -1.88600006e+02, -1.88434067e+02]],

       [[-2.51518921e+02, -2.50241302e+02, -2.48906998e+02, ...,
         -2.07415512e+02, -2.06461365e+02, -2.05522919e+02],
        [-3.06119141e+02, -3.02657898e+02, -2.99096252e+02, ...,
         -3.90520172e+02, -3.91377228e+02, -3.92164581e+02]],

       [[ 1.31926721e+03,  1.31964050e+03,  1.32002942e+03, ...,
          1.30337354e+03,  1.30373279e+03,  1.30409521e+03],
        [ 1.37926257e+03,  1.37985828e+03,  1.38050366e+03, ...,
          1.31928870e+03,  1.31982031e+03,  1.32035034e+03]],

       [[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.0000

In [5]:
def read_labels_base(data_path,frame_rate):  

    """

    Funkcja zwraca tablice [p, k], w której są zapisane czasy eventow oznaczających przyjecie postawy poczatkowej.

	

	Input:

	- data_path - sciezka do pliku c3d

	- frame_rate - częstotliwośc próbkowania danych w pliku



	Output:

    - [p,k] - tablice punktów startowych (s) i końcowych(k)


    """
    c3d_to_compare= c3d(data_path)
    event = c3d_to_compare['parameters']['EVENT']['LABELS']['value']
    czas = np.around(c3d_to_compare['parameters']['EVENT']['TIMES']['value'][1]*frame_rate)
    eventy = [event, czas]


    eventy[0].index('Foot Strike')
    indxFS = [i for i, x in enumerate(eventy[0]) if x == "Foot Strike"]
    CzasFS = eventy[1][indxFS[0]]   
    indxE = [i for i, x in enumerate(eventy[0]) if x == "Event"]
    CzasE= eventy[1][indxE[0]]
    
    return CzasFS-CzasE

In [6]:
event=read_labels_base(data_path,200)
#print(event)

In [7]:
data=[]
for marker in range(len(markers_names)):
    data.append([])
    data[marker].append([markers[0][marker][0].values,markers[1][marker][0].values,markers[2][marker][0].values])
    data[marker].append([markers[0][marker][event.astype(int)].values,markers[1][marker][event.astype(int)].values,markers[2][marker][event.astype(int)].values])
    data[marker].append([markers[0][marker][-1].values,markers[1][marker][-1].values,markers[2][marker][-1].values])

In [28]:
def load_data(data_path):
    event=read_labels_base(data_path,200)
    data=[]
    for marker in range(len(markers_names)):
        data.append([])
        data[marker].append([markers[0][marker][0].values,markers[1][marker][0].values,markers[2][marker][0].values])
        data[marker].append([markers[0][marker][event.astype(int)].values,markers[1][marker][event.astype(int)].values,markers[2][marker][event.astype(int)].values])
        data[marker].append([markers[0][marker][-1].values,markers[1][marker][-1].values,markers[2][marker][-1].values])
    return data

In [55]:
data_path1=r"Z:\baza\2020-08-05-B0444-S01-E01-T01-K01.c3d"
data_path2=r"Z:\baza\2020-08-05-B0444-S01-E02-T01-K01.c3d"
data_path3=r"Z:\baza\2020-08-05-B0444-S01-E03-T01-K01.c3d"
data_path4=r"Z:\baza\2020-08-05-B0444-S01-E04-T01-K02.c3d"
data_path5=r"Z:\baza\2020-08-05-B0444-S01-E05-T01-K01.c3d"
data.append(load_data(data_path1))
data.append(load_data(data_path2))
data.append(load_data(data_path3))
data.append(load_data(data_path4))
data.append(load_data(data_path5))


In [27]:
from pomegranate import *

model = HiddenMarkovModel()
d1 = DiscreteDistribution({'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25})
d2 = DiscreteDistribution({'A': 0.10, 'C': 0.40, 'G': 0.40, 'T': 0.10})


s1 = State(d1, name='background')
s2 = State(d2, name='CG island')

model = HiddenMarkovModel()
model.add_states(s1, s2)
model.add_transition(model.start, s1, 0.5)
model.add_transition(model.start, s2, 0.5)
model.add_transition(s1, s1, 0.9)
model.add_transition(s1, s2, 0.1)
model.add_transition(s2, s1, 0.1)
model.add_transition(s2, s2, 0.9)
model.bake()
model.predict(d1)
print(model.probability(list('background')))

TypeError: len() of unsized object

In [81]:
from pomegranate import *

model = HiddenMarkovModel()
d1 = DiscreteDistribution({'S': 1/3, 'K':1/3, '1': 1/3, '2': 0, '3': 0, '4': 0, '5': 0.})
d2 = DiscreteDistribution({'S': 1/3, 'K': 1/3, '1': 0, '2': 1/3, '3': 0, '4': 0, '5': 0})
d3 = DiscreteDistribution({'S': 1/3, 'K': 1/3, '1': 0, '2': 0, '3': 1/3, '4': 0, '5': 0})
d4 = DiscreteDistribution({'S': 1/3, 'K': 1/3, '1': 0, '2': 0, '3': 0, '4': 1/3, '5': 0})
d5 = DiscreteDistribution({'S': 1/3, 'K': 1/3, '1': 0, '2': 0, '3': 0, '4': 0, '5': 1/3})

ds = DiscreteDistribution({'S': 1/3, 'K': 1/3, '1': 0, '2': 0, '3': 0, '4': 1/3, '5': 0})
dk = DiscreteDistribution({'S': 1/3, 'K': 1/3, '1': 0, '2': 0, '3': 0, '4': 0, '5': 1/3})

s1 = State(d1, name='Cwiczenie 1')
s2 = State(d2, name='Cwiczenie 2')
s3 = State(d3, name='Cwiczenie 3')
s4 = State(d4, name='Cwiczenie 4')
s5 = State(d5, name='Cwiczenie 5')

s = State(ds, name='Pozycja poczatkowa')
k = State(dk, name='Pozycja koncowa')

model = HiddenMarkovModel()
model.add_states(s1, s2, s3, s4, s5, s, k)

model.add_transition(model.start, s1, 0.0)
model.add_transition(model.start, s2, 0.0)
model.add_transition(model.start, s3, 0.0)
model.add_transition(model.start, s4, 0.0)
model.add_transition(model.start, s5, 0.0)

model.add_transition(model.start, s, 1.0)
model.add_transition(model.start, k, 0.0)

model.add_transition(s, k, 0.0)
model.add_transition(k, s, 0.0)

model.add_transition(s1, k, 1)
model.add_transition(s2, k, 1)
model.add_transition(s3, k, 1)
model.add_transition(s4, k, 1)
model.add_transition(s5, k, 1)

model.add_transition(s, s1, 0.2)
model.add_transition(s, s2, 0.2)
model.add_transition(s, s3, 0.2)
model.add_transition(s, s4, 0.2)
model.add_transition(s, s5, 0.2)

model.add_transition(k, s1, 1)
model.add_transition(k, s2, 1)
model.add_transition(k, s3, 1)
model.add_transition(k, s4, 1)
model.add_transition(k, s5, 1)

model.bake()

print(model.probability("S1K"))

0.007407407407407406


In [84]:
1/5*1/3*1/3*1/3

0.007407407407407408

In [82]:
0.022222222222222223/0.007407407407407406

3.000000000000001

In [99]:
from pomegranate import *

model = HiddenMarkovModel()




d1 = DiscreteDistribution({'S': 0, 'K':0, '1':1, '2': 0, '3': 0, '4': 0, '5': 0.})
d2 = DiscreteDistribution({'S': 0, 'K': 0, '1': 0, '2': 1, '3': 0, '4': 0, '5': 0})
d3 = DiscreteDistribution({'S': 0, 'K': 0, '1': 0, '2': 0, '3':1, '4': 0, '5': 0})
d4 = DiscreteDistribution({'S': 0, 'K': 0, '1': 0, '2': 0, '3': 0, '4': 1, '5': 0})
d5 = DiscreteDistribution({'S': 0, 'K': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 1})

ds = DiscreteDistribution({'S': 1, 'K': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0})
dk = DiscreteDistribution({'S': 0, 'K':1, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0})

s1 = State(d1, name='Cwiczenie 1')
s2 = State(d2, name='Cwiczenie 2')
s3 = State(d3, name='Cwiczenie 3')
s4 = State(d4, name='Cwiczenie 4')
s5 = State(d5, name='Cwiczenie 5')

s = State(ds, name='Pozycja poczatkowa')
k = State(dk, name='Pozycja koncowa')

model = HiddenMarkovModel()
model.add_states(s1, s2, s3, s4, s5, s, k)

model.add_transition(model.start, s1, 0.0)
model.add_transition(model.start, s2, 0.0)
model.add_transition(model.start, s3, 0.0)
model.add_transition(model.start, s4, 0.0)
model.add_transition(model.start, s5, 0.0)

model.add_transition(model.start, s, 1.0)
model.add_transition(model.start, k, 0.0)

model.add_transition(s, k, 0.0)
model.add_transition(k, s, 0.0)

model.add_transition(s1, k, 1)
model.add_transition(s2, k, 1)
model.add_transition(s3, k, 1)
model.add_transition(s4, k, 1)
model.add_transition(s5, k, 1)

model.add_transition(s, s1, 0.2)
model.add_transition(s, s2, 0.2)
model.add_transition(s, s3, 0.2)
model.add_transition(s, s4, 0.2)
model.add_transition(s, s5, 0.2)

model.add_transition(k, s1, 1)
model.add_transition(k, s2, 1)
model.add_transition(k, s3, 1)
model.add_transition(k, s4, 1)
model.add_transition(k, s5, 1)

model.bake()

print(model.probability("S1"))

0.2
